In [43]:
import os
import shutil

def move_xls_files(big_folder, metadata_folder):
    """
    Moves .xls files from a specified 'big_folder' to a 'metadata_folder'.
    Both folders are assumed to be relative to the current working directory.

    Parameters:
    - big_folder: The relative path to the folder containing .xlsx files.
    - metadata_folder: The relative path to the folder where .xlsx files should be moved.
    """

    # Ensure the metadata folder exists, if not, create it
    if not os.path.exists(metadata_folder):
        os.makedirs(metadata_folder)

    # List all files in the big folder
    for file in os.listdir(big_folder):
        # Check if the file ends with .xls
        if file.endswith('.xlsx'):
            # Construct the full path to the file using the relative path
            file_path = os.path.join(big_folder, file)
            
            # Construct the destination path in the metadata folder using the relative path
            dest_path = os.path.join(metadata_folder, file)
            
            # Move the .xls file to the metadata folder
            shutil.move(file_path, dest_path)
            print(f"Moved: {file_path} to {dest_path}")

# Example usage

big_folder_path = 'garden_01012024/'  # Adjust if your folder has a different name
metadata_folder_path = 'metadata'
move_xls_files(big_folder_path, metadata_folder_path)


In [ ]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()
afile = "cardinal.mp3"
bfile = "er_file_2023_09_04_7_13_11.wav"
cfile = "er_file_2023_09_04_7_13_08.wav"
dfile = "er_file_2023_09_04_7_39_44.wav"
efile = "er_file_2023_09_04_7_49_38.wav"
ffile = "er_file_2023_09_04_8_24_14.wav"
recording = Recording(
    analyzer,
    bfile,
    lat=51.41778,
    lon=5.462251,
    date=datetime(year=2022, month=5, day=10), # use date or week_48
    min_conf=0.25,
)
recording.analyze()
print(recording.detections)

In [8]:
recording.detections[0]['label']

'Streptopelia decaocto_Eurasian Collared-Dove'

In [77]:
#analyze files form folder and save results in a tuple: file name, label
import os
def analyze_files_birdnet(folder):
    """
    Analyzes all files in a specified folder using the BirdNET-Analyzer and returns the results.
    The results are a list of tuples, where each tuple contains the file name and the detected label.
    """    
    files = os.listdir(folder)
    results = []
    for file in files:
        try:
            recording = Recording(
                analyzer,
                folder + file,
                lat=51.41778,
                lon=5.462251,
                date=datetime(year=2022, month=5, day=10), # use date or week_48
                min_conf=0.25,
            )
            recording.analyze()
            if len(recording.detections) > 0:
                results.append((folder + file, recording.detections[0]['label']))
        except Exception as e:
            print(f"Error processing file {file}: {e}")
    return results

In [44]:
from tqdm import tqdm
# analyze files in the folder of folders, wrap in tqdm to show progress bar

def analyze_folders(folder):
    """ 
    Analyzes all folders in a specified folder using the BirdNET-Analyzer and returns the results.
    The results are a list of tuples, where each tuple contains the file name and the detected label.
    """
    folders = [f for f in os.listdir(folder) if not f.startswith('.') and not f.endswith('.json')]
    results = []
    for f in tqdm(folders):
        results += analyze_files_birdnet(folder + f + "/")
       
    return results

In [30]:
import pandas as pd

# method to add a column with 'label' to the dataframe, allign by file name
def add_label_column(df, results):
    """
    Adds a 'label' column to a DataFrame based on the results of the BirdNET-Analyzer.
    The results are a list of tuples, where each tuple contains the file name and the detected label.
    """
    df['label'] = None
    for file, label in results:
        df.loc[df['filename'] == file, 'label'] = label
    return df


In [24]:
# get metadata_file and output_file from folder name
def get_metadata_output_files(folder):
    """
    Returns the metadata file and output file based on the folder name.
    The metadata file is assumed to be in the 'metadata' folder with the same name as the folder.
    The output file is assumed to be in the 'metadata' folder with the same name as the folder,
    but with '_birdnet_labeled' appended before the file extension.
    """
    metadata_file = "metadata/" +folder[:-1] + "_metadata.xlsx"
    output_file = metadata_file.split(".")[0] + "_birdnet_labeled.xlsx"
    return metadata_file, output_file

folder = "garden_02092023/"
metadata_file, output_file = get_metadata_output_files(folder)
metadata_file, output_file

('metadata/garden_02092023_metadata.xlsx',
 'metadata/garden_02092023_metadata_birdnet_labeled.xlsx')

In [31]:
# save the dataframe to a new xlsx file
df.to_excel("garden_02092023_metadata_with_labels.xlsx")


In [26]:
def pipeline_birdnet(folder):
    """
    Runs the full pipeline for analyzing a folder of audio files using the BirdNET-Analyzer.
    The pipeline consists of the following steps:
    1. Move .xls files to a metadata folder.
    2. Analyze the folders using the BirdNET-Analyzer.
    3. Add the detected labels to the metadata file.
    4. Save the updated metadata file with the labels.
    """
    move_xls_files(folder, metadata_folder_path)
    metadata_file, output_file = get_metadata_output_files(folder)
    results = analyze_folders(folder)
    df = pd.read_excel(metadata_file)
    df = add_label_column(df, results)
    df.to_excel(output_file)

In [ ]:
# test the pipeline
folder = "garden_11082023/"
metadata_folder_path = 'metadata'

pipeline_birdnet(folder)


In [83]:
mf = pd.read_excel("metadata/garden_21102023_metadata_birdnet_labeled.xlsx")
mf[mf['label'].notnull()].shape

(1950, 26)